In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import precision_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
test_df = pd.read_csv('./kaggle_data/test.csv')
train_df = pd.read_csv('./kaggle_data/train.csv')

Need to better understand `spray_df` and identify where weather stations 1 and 2 are.  All dataframes include date data, so the next step is to figure out how to appropriately combine them.

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
#sns.heatmap(train_df.corr(), annot=True)

Unsurprisingly, the likelihood of WNV increases as the number of mosquitos increase.

In [5]:
#print(train_df.shape)
#train_df.isnull().sum()

In [6]:
#print(spray_df.shape)
#spray_df.isnull().sum()
                 ## We will need to deal with these time nulls, but it may make sense to drop the time column
                  # since the other dfs dont have time

In [7]:
#print(spray_df.shape)
#weather_df.isnull().sum()

In [9]:
#spray_df.head()

test_df['Date'] = pd.to_datetime(test_df['Date']) #datetime index on training data
train_df['Date'] = pd.to_datetime(train_df['Date'])
test_df.set_index('Date', inplace=True, drop=True)
train_df.set_index('Date', inplace = True, drop = True)

In [10]:
train_df['Date'] = pd.to_datetime(train_df['Date'])
test_df['Date'] = pd.to_datetime(train_df['Date'])

In [11]:
#new_df.shape, train_df.shape, weather_df.shape

In [17]:
new_train_df = train_df
new_test_df = test_df

In [24]:
wnv1_df = new_train_df[new_train_df['WnvPresent'] == 1]

In [25]:
wnv0_df = new_train_df[new_train_df['WnvPresent'] == 0].sample(n = wnv1_df.shape[0], random_state = 42)

In [26]:
train_df = pd.concat([wnv1_df, wnv0_df], axis = 0)

In [27]:
num_df = train_df._get_numeric_data()
num_test_df = new_test_df._get_numeric_data()

In [28]:
X = num_df.drop(['WnvPresent', 'NumMosquitos'], axis = 1)
y = num_df.WnvPresent

In [29]:
ids = num_test_df.Id

In [30]:
num_test_df.drop('Id', axis=1, inplace=True)

In [31]:
ss = StandardScaler()

In [32]:
ss.fit(X)
test_num = ss.transform(num_test_df)

In [33]:
ss = ss.transform(X)
X = pd.DataFrame(ss, columns=X.columns)


In [34]:
pca = PCA()

In [35]:
pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [36]:
Z = pca.transform(X)
ZTEST = pca.transform(num_test_df)

In [37]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

## Random Forrest on Numerical Data

In [38]:
num_test_df

,Block,Latitude,Longitude,AddressAccuracy
0,41,41.954690,-87.800991,9
1,41,41.954690,-87.800991,9
2,41,41.954690,-87.800991,9
3,41,41.954690,-87.800991,9
4,41,41.954690,-87.800991,9
5,41,41.954690,-87.800991,9
6,41,41.954690,-87.800991,9
7,41,41.954690,-87.800991,9
8,62,41.994991,-87.769279,9
9,62,41.994991,-87.769279,9


In [39]:
rf = RandomForestClassifier(n_jobs=4)
rf_params = {'n_estimators':[8,10,12,14,20],
            'min_samples_split':[2,3,4],
            'random_state':[1,21,31,100]}
rf_gs = GridSearchCV(rf, rf_params, n_jobs=4)
rf_gs.fit(Z, y)
rf_gs.best_params_

{'min_samples_split': 3, 'n_estimators': 14, 'random_state': 1}

In [40]:
rf_preds = rf_gs.predict(ZTEST)

In [41]:
submission = pd.DataFrame({'Id':ids, 'WnvPresent':rf_preds})

In [42]:
submission.set_index('Id', drop=True, inplace=True)

In [43]:
submission.WnvPresent.value_counts()

1    116293
Name: WnvPresent, dtype: int64

In [ ]:
len(rf_preds), len(X_test)

In [ ]:
pred_df = pd.DataFrame({'preds':rf_preds})

In [ ]:
X_test = ss.inverse_transform(X_test)

In [ ]:
X_test = pd.DataFrame(X_test, columns=X.columns)

In [ ]:
pred_df = pd.concat([X_test[['Longitude','Latitude']], pred_df], axis = 1)

In [ ]:
wnv_locs = pred_df[pred_df['preds'] == 1]

In [ ]:
wnv_locs

## AdaBoost on Numerical Data

In [ ]:
ada = AdaBoostClassifier()
ada_params = {'n_estimators':[30,40,50,60,80],
             'learning_rate':[.8,.4,.1],}
ada_gs = GridSearchCV(ada, ada_params, n_jobs=4)
ada_gs.fit(Z_train, y_train)

In [ ]:
ada_preds = ada_gs.predict(Z_test)

In [ ]:
ada_gs.best_params_

In [ ]:
precision_score(y_test, ada_preds), accuracy_score(y_test, ada_preds)

## Gradient Boost on Numerical Data

In [ ]:
xgb = GradientBoostingClassifier()
xgb_params = {'learning_rate':[.01, .05, .1, .3, .5],
             'n_estimators':[50,80, 100, 120],
             'min_samples_split':[2,3,4]}
xgb_gs = GridSearchCV(xgb, xgb_params, n_jobs=4)
xgb_gs.fit(Z_train, y_train)

In [ ]:
xgb_gs.best_params_

In [ ]:
xgb_preds = xgb_gs.predict(Z_test)

In [ ]:
precision_score(y_test, xgb_preds), accuracy_score(y_test, xgb_preds)

## Trying same three models on dummied data

In [ ]:
df.columns

In [ ]:
dummies_df = pd.get_dummies(df.drop(['Address','Street'], axis = 1))

In [ ]:
X_dum = dummies_df.drop('WnvPresent', axis = 1)
y = dummies_df.WnvPresent

In [ ]:
Xdum_train, Xdum_test, ydum_train, ydum_test = train_test_split(X_dum, y, test_size = .25)

In [ ]:
ss_dum = StandardScaler()
ss_dum.fit(Xdum_train)

In [ ]:
Xdum_train = ss_dum.transform(Xdum_train)
Xdum_test = ss_dum.transform(Xdum_test)

In [ ]:
pca_dum = PCA(.95)

In [ ]:
pca_dum.fit(Xdum_train)

In [ ]:
Zdum_train = pca_dum.transform(Xdum_train)
Zdum_test = pca_dum.transform(Xdum_test)

## Random Forrest on Dummy Data
### Random forest had highest accuracy of 78.6% but lower precision than AdaBoost using just numerical data

In [ ]:
rf_dum = RandomForestClassifier(n_jobs=4)
rf_params = {'n_estimators':[8,10,12,14,20],
            'min_samples_split':[2,3,4],
            'random_state':[1,21,31,100]}
rf_dum_gs = GridSearchCV(rf_dum, rf_params, n_jobs=4)
rf_dum_gs.fit(Zdum_train, ydum_train)
rf_dum_gs.best_params_

In [ ]:
rf_dum_preds = rf_dum_gs.predict(Zdum_test)

In [ ]:
precision_score(ydum_test, rf_dum_preds), accuracy_score(ydum_test, rf_dum_preds)

## AdaBoost on Dummy Data

In [ ]:
ada_dum = AdaBoostClassifier()
ada_dum_params = {'n_estimators':[30,40,50,60,80],
             'learning_rate':[.8,.4,.1],}
ada_dum_gs = GridSearchCV(ada_dum, ada_params, n_jobs=4)
ada_dum_gs.fit(Zdum_train, ydum_train)

In [ ]:
ada_dum_preds = ada_dum_gs.predict(Zdum_test)

In [ ]:
ada_dum_gs.best_params_

In [ ]:
precision_score(ydum_test, ada_dum_preds), accuracy_score(ydum_test, ada_dum_preds)

## Gradient Boost on Dummy Data

In [ ]:
xgb_dum = GradientBoostingClassifier()
xgb_dum_params = {'learning_rate':[.01, .05, .1, .3, .5],
             'n_estimators':[50,80, 100, 120]}
xgb_dum_gs = GridSearchCV(xgb_dum, xgb_dum_params, n_jobs=4)
xgb_dum_gs.fit(Zdum_train, ydum_train)

In [ ]:
xgb_dum_gs.best_params_

In [ ]:
xgb_dum_preds = xgb_dum_gs.predict(Zdum_test)

In [ ]:
precision_score(ydum_test, xgb_dum_preds), accuracy_score(ydum_test, xgb_dum_preds)

# Suzanne Pipeline

In [ ]:
X = df.drop(columns = ['WnvPresent', 'Address', 
                           'Species', 'Street', 'Trap', 
                           'AddressNumberAndStreet', 'WetBulb'
                           , 'Heat', 'CodeSum'],
                            axis = 1)
y = df['WnvPresent']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size = 0.33, 
                                                   random_state = 42)

# Instantiating Preprocessing and Model 
lr = LogisticRegression()
ss = StandardScaler()

# Setting up Pipeline 

lr_pipe = Pipeline([
    ('ss', ss),
    ('lr', lr)
])

# Setting up Parameter Dictionary 
gs_lr_params = {
    'lr__penalty': ['l1', 'l2'], 
    'lr__C': [0.5, 1.0, 1.2]
}

# Instantiating and Fitting my Grid Search
gs_lr = GridSearchCV(lr_pipe, param_grid=gs_lr_params)
gs_lr.fit(X_train, y_train);

print("Best Params:", gs_lr.best_params_)
print("Best Train Score:", gs_lr.best_score_ )
print("Best Test Score:", gs_lr.score(X_test, y_test) )